In [1]:
from arcgis.gis import GIS
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools

%load_ext autoreload
%autoreload 2

In [2]:
url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

reach_id_ldub = 2156
reach_id_farmies = 2269
reach_id_truss = 2270
reach_id_bz = 3064
reach_id_canyon = 3066
reach_id_opal = 5199

lst_reach_id = [reach_id_ldub, reach_id_farmies, reach_id_truss, reach_id_bz, reach_id_canyon, reach_id_opal]

In [3]:
gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))
gis

GIS @ https://knu2xs.maps.arcgis.com version:6.4

In [4]:
lyr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

In [ ]:
processed_df = lyr_reach_points.query(out_fields='reach_id', return_geometry=False).sdf
unprocessed_reach_id_lst = [val for val in range(1, 6001) if val not in processed_df.reach_id.astype('int64').values]

invalid_lst = []

for reach_id in unprocessed_reach_id_lst:
    try:
        reach = reach_tools.Reach.get_from_aw(reach_id)
        reach.update_putin_takeout_and_trace()
        reach.publish(gis, lyr_reach_line, lyr_reach_centroid, lyr_reach_points)
        print('published', reach.reach_id, '-', reach.river_name, reach.reach_name)
    except:
        invalid_lst.append(reach.reach_id)
        print('Could not publish', reach.reach_id, '-', reach.river_name, reach.reach_name)

published 438 - Kennebec 1) Kennebec Gorge: Harris Station Dam to Carry Brook
published 635 - Snake A) Palisades Dam to Launch Site East of Poplar (Swan Valley)
published 636 - St. Joe 1\. Upper: Heller Creek to Spruce Tree Campground
published 637 - St. Joe 2\. Spruce Tree Camp to Gold Creek
published 638 - St. Joe 3\. Tumbledown: Conrad Crossing to Bluff Creek
published 639 - St. Joe 4\. Skookum Canyon: Turner Creek to Packsaddle
published 640 - St. Joe, N. Fork 1\. Loop Creek to St. Joe Confluence
published 641 - St. Maries Mashburn to St. Joe River
published 642 - Clackamas, South Fork South Fork Clackamas Falls
published 643 - Teton Teton Dam site to Teton Forks
published 644 - Warm Twisted Draw to Warm River Campground
published 645 - Colt Killed Creek (White Sand Creek) 1\. Big Flat Creek to Colt Killed Creek Trailhead
published 646 - Colt Killed Creek (White Sand Creek) 2\. Colt Killed Creek Trailhead to Lochsa River
published 647 - Wildhorse Bear Creek Bridge to Snake River co

In [ ]:
lyr_reach_line.flush()
lyr_reach_centroid.flush()
lyr_reach_points.flush()

In [15]:
reach = reach_tools.Reach.get_from_aw(2270)
webmap = reach.update_putin_takeout_and_trace(webmap=True)
webmap

MapView(layout=Layout(height='400px', width='100%'))